In [6]:
# Module for predicting top n best possible word matches given a word

# Context:
# A -> True word
# B -> Typed word

# Make dictionary, key (true word), value (list of incorrect words)
# tru_in dict is A -> [B1,B2,...,Bn]
missp_dat_path = "/Users/daveistanto/Documents/project_normal/advanced_functions/predictive_search/dataset/missp.dat"
wiki_dat_path = "/Users/daveistanto/Documents/project_normal/advanced_functions/predictive_search/dataset/wikipedia.dat"
aspell_dat_path = "/Users/daveistanto/Documents/project_normal/advanced_functions/predictive_search/dataset/aspell.dat"

tru_in = dict()

def add_to_dict(data_path, input_dict):
    with open(data_path, 'r') as df:
        data_lines = df.readlines()
        for line in data_lines:
            lns = line[:-1].upper()
            if lns[0] == "$":     
                curr_key = lns[1:]
                if curr_key not in list(set(input_dict)):
                    input_dict[curr_key] = []
            else:
                 input_dict[curr_key].append(lns)
    
    for k, v in input_dict.items():
        tru_in[k] = list(set(tru_in[k]))
    return input_dict

tru_in = add_to_dict(missp_dat_path, tru_in)
tru_in = add_to_dict(wiki_dat_path, tru_in)
tru_in = add_to_dict(aspell_dat_path, tru_in)


In [7]:
# Store dictionary in json

import json

with open('../saved_states/tru_in.json', 'w') as fp:
    json.dump(tru_in, fp)

In [4]:
# Get P(A) distribution

import pandas as pd
import numpy as np
word_freq_path = "/Users/daveistanto/Documents/project_normal/advanced_functions/predictive_search/dataset/top_5k_words.csv"
word_freq_df = pd.read_csv(word_freq_path)

def normalize_word(input_word):
    input_word = input_word.upper()[3:]
    return input_word

normalize_word = np.vectorize(normalize_word)

word_freq_df["Normalized word"] = np.apply_along_axis(normalize_word, 0, word_freq_df["Word"].values)
word_freq_df = word_freq_df.drop(columns=["Word"])
word_freq_df["Relative Frequency"] = 0

# Add dummy frequency of 1000 if word not found
for k in tru_in.keys():
    if k.upper() not in word_freq_df["Normalized word"].values:
        new_row = {"Frequency": 1000, "Normalized word": k}
        word_freq_df = word_freq_df.append(new_row, ignore_index=True)

# Add relative frequency (P(A)) for each A
word_sum = word_freq_df["Frequency"].sum()
for row_index in range(len(word_freq_df)):
    word_freq_df.iloc[row_index, 2] = word_freq_df.iloc[row_index, 0] / word_sum
    
    
# Sort word_freq_df by frequency
word_freq_df = word_freq_df.sort_values(by="Frequency", ascending=False)


In [13]:
# Store word_freq_df in csv
word_freq_df.to_csv('/Users/daveistanto/Documents/project_normal/advanced_functions/predictive_search/saved_states/word_freq_df.csv', index=False)

In [1]:
# Load word_freq_df and tru_in dictionary
import pandas as pd
import json

word_freq_df = pd.read_csv("../saved_states/word_freq_df.csv")

with open('../saved_states/tru_in.json', 'r') as fp:
    tru_in = json.load(fp)

In [2]:
# Get P(B|A) distribution for specific B. Does not equal 1 because different A(s).

def get_P_BA(B):
    PBA_dict = dict()
    B = B.upper()
    for k, v in tru_in.items():
        PBA_for_given_A = tru_in[k].count(B) / len(tru_in[k])
        PBA_dict[k] = PBA_for_given_A
        
    return PBA_dict

In [3]:
# Get P(A) given specific A:

def get_PA(A):
    P_A = word_freq_df["Relative Frequency"][word_freq_df["Normalized word"] == A].values[0]
    return P_A
    

In [4]:
# Get relative P(A|B)

def get_PAB(B):
    B = B.upper()
    PBA_dict = get_P_BA(B)
    PAB_list = []
    for k, v in PBA_dict.items():
            if v != 0:
                P_A = get_PA(k)
                tup = (k, PBA_dict[k] * P_A)
                PAB_list.append(tup)

    PAB_list.append((B, 1))
    
    # Sort by value, descending
    PAB_list.sort(key=lambda tup: tup[1], reverse=True)
    
    #Get the words only
    words_only = [i[0] for i in PAB_list]
    
    return words_only

print(get_PAB("are"))


['ARE', 'A', 'AND', 'I', 'OR', 'AREA', 'AIR', 'OUR', 'EYE', 'FIRE', 'ERE']


In [7]:
import sys
print(get_PAB(sys.argv[1]))

['-F']
